In [1]:
import tensorflow as tf
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

ModuleNotFoundError: No module named 'tensorflow'

In [13]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [14]:
x_train = x_train/ 255.0
x_test = x_test / 255.0

In [37]:
x_train_reshape = tf.reshape(x_train, shape=(-1, 28*28))
x_test_reshape = tf.reshape(x_test, shape=(-1, 28*28))

In [2]:
class exp_activation(tf.keras.layers.Layer):
    def __init__(self):
        super(exp_activation, self).__init__()
    
    def call(self, input_tensor):
        return tf.math.exp(tf.cast(input_tensor, dtype=tf.float32))

In [23]:
class myrelu(tf.keras.layers.Layer):
    def __init__(self):
        super(myrelu, self).__init__()
        
    def call(self, x):
        return tf.math.maximum(x, 0)

In [3]:
exp_activation()(tf.Variable([[-23, 23, 435, -23, -12453, -45],
                      [-123, 23, 34, -1, 23, 36]]))

<tf.Tensor: shape=(2, 6), dtype=float32, numpy=
array([[1.02618795e-10, 9.74480282e+09,            inf, 1.02618795e-10,
        0.00000000e+00, 2.86251861e-20],
       [0.00000000e+00, 9.74480282e+09, 5.83461711e+14, 3.67879480e-01,
        9.74480282e+09, 4.31123180e+15]], dtype=float32)>

In [4]:
class relu_exp_activation(tf.keras.layers.Layer):
    def __init__(self):
        super(relu_exp_activation, self).__init__()
    
    def call(self, input_tensor):
        return tf.math.maximum(tf.math.exp(tf.cast(input_tensor, dtype=tf.float32)), 0.15)

In [5]:
relu_exp_activation()(tf.Variable([[-23, 23, 435, -23, -12453, -45],
                      [-123, 23, 34, -1, 23, 36]]))

<tf.Tensor: shape=(2, 6), dtype=float32, numpy=
array([[1.5000001e-01, 9.7448028e+09,           inf, 1.5000001e-01,
        1.5000001e-01, 1.5000001e-01],
       [1.5000001e-01, 9.7448028e+09, 5.8346171e+14, 3.6787948e-01,
        9.7448028e+09, 4.3112318e+15]], dtype=float32)>

In [34]:
class myModel(tf.keras.Model):
    def __init__(self, numofclasses):
        super(myModel, self).__init__()
        self.dense = tf.keras.layers.Dense(64)
        self.classifier = tf.keras.layers.Dense(numofclasses)
        self.exp = exp_activation()
    
    def call(self, input_tensor):
        x = self.dense(input_tensor)
        x = self.exp(x)
        x = self.classifier(x)
        x = tf.nn.softmax(x)
        return x

In [48]:
test_model = myModel(10)

test_model.compile(loss = tf.keras.losses.sparse_categorical_crossentropy,
                  optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
                  metrics = ['accuracy'])
test_model.fit(x=x_train_reshape, y=y_train, batch_size=32, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2741 - accuracy: 0.9222
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1325 - accuracy: 0.9616
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1029 - accuracy: 0.9694
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0826 - accuracy: 0.9747
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0769 - accuracy: 0.9774


In [49]:
test_model.evaluate(x_test_reshape, y_test, batch_size=32)

313/313 [==============================] - 1s 2ms/step - loss: 0.1155 - accuracy: 0.9697


[0.11546734720468521, 0.9696999788284302]

In [12]:
class myModel_2(tf.keras.Model):
    def __init__(self, numofclasses):
        super(myModel_2, self).__init__()
        self.dense = tf.keras.layers.Dense(64)
        self.classifier = tf.keras.layers.Dense(numofclasses)
        self.exp = exp_activation()
    
    def call(self, input_tensor):
        x = self.dense(input_tensor)
        x = self.exp(x)
        x = self.classifier(x)
        x = tf.nn.softmax(x)
        return x

In [46]:
test_model_2 = myModel_2(10)

test_model_2.compile(loss = tf.keras.losses.sparse_categorical_crossentropy,
                  optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
                  metrics = ['accuracy'])
test_model_2.fit(x=x_train_reshape, y=y_train, batch_size=32, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2733 - accuracy: 0.9225
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1321 - accuracy: 0.9618
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0988 - accuracy: 0.9703
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0834 - accuracy: 0.9751
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0737 - accuracy: 0.9785


In [47]:
test_model_2.evaluate(x_test_reshape, y_test, batch_size=32)

313/313 [==============================] - 1s 3ms/step - loss: 0.1221 - accuracy: 0.9706


[0.12210167199373245, 0.9706000089645386]

In [15]:
class CNNBlock(tf.keras.Model):
    def __init__(self, units, kernel_size = 3):
        super(CNNBlock, self).__init__()
        self.conv = tf.keras.layers.Conv2D(units, kernel_size, padding='same')
        self.bn = tf.keras.layers.BatchNormalization()
        self.activation = exp_activation()
    
    def call(self, input_tensor, training=False):
        x = self.conv(input_tensor)
        x = self.bn(x, training=training)
        x = self.activation(x)
        return x


In [28]:
class cnnmodel(tf.keras.Model):
    def __init__(self, numofclasses):
        super(cnnmodel, self).__init__()
        self.cnnblock1 = CNNBlock(32)
        self.cnnblock2 = CNNBlock(64)
        self.cnnblock3 = CNNBlock(128)
        self.exp = exp_activation()
        self.pooling = tf.keras.layers.MaxPool2D()
        self.flatten = tf.keras.layers.Flatten()
        self.dense = tf.keras.layers.Dense(64)
        self.classifier = tf.keras.layers.Dense(numofclasses, activation='softmax')
    
    def call(self, input_tensor, training=False):
        x = tf.reshape(input_tensor, shape=(-1, 32, 32, 3))
        x = self.cnnblock1(x, training=training)
        x = self.cnnblock1(x, training=training)
        x = self.pooling(x)
        
        x = self.cnnblock2(x, training=training)
        x = self.cnnblock2(x, training=training)
        x = self.pooling(x)
        
        x = self.cnnblock3(x, training=training)
        x = self.cnnblock3(x, training=training)
        x = self.cnnblock3(x, training=training)
        x = self.pooling(x)
        
        x = self.flatten(x)
        x = self.dense(x)
        x = self.exp(x)
        x = self.dense(x)
        x = self.exp(x)
        x = self.classifier(x)
        return x

In [7]:
(x_train_2, y_train_2), (x_test_2, y_test_2) = tf.keras.datasets.cifar10.load_data()

In [8]:
x_train_2 = x_train_2 / 255.0
x_test_2 = x_test_2 / 255.0

In [23]:
x_train_2.shape

(50000, 32, 32, 3)

In [29]:
cnn_model = cnnmodel(10)

cnn_model.compile(loss = tf.keras.losses.sparse_categorical_crossentropy,
                 optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
                 metrics = ['accuracy'])

cnn_model.fit(x=x_train_2, y=y_train_2, batch_size=32, epochs=10)

Epoch 1/10


ValueError: in user code:

    C:\Users\An_asus\Anaconda3\envs\tf\lib\site-packages\tensorflow\python\keras\engine\training.py:855 train_function  *
        return step_function(self, iterator)
    <ipython-input-28-06736dd97ec1>:16 call  *
        x = self.cnnblock1(x, training=training)
    <ipython-input-15-2c8c439315aa>:9 call  *
        x = self.conv(input_tensor)
    C:\Users\An_asus\Anaconda3\envs\tf\lib\site-packages\tensorflow\python\keras\engine\base_layer.py:1013 __call__  **
        input_spec.assert_input_compatibility(self.input_spec, inputs, self.name)
    C:\Users\An_asus\Anaconda3\envs\tf\lib\site-packages\tensorflow\python\keras\engine\input_spec.py:251 assert_input_compatibility
        raise ValueError(

    ValueError: Input 0 of layer conv2d_17 is incompatible with the layer: expected axis -1 of input shape to have value 3 but received input with shape (None, 32, 32, 32)
